### 读入tools 生成的possible matchings

In [128]:

matchings_path = "./dataset/employee.json"
answers_path = "./dataset/employee_ans.json"
matchings_path = "./dataset/author.json"
answers_path = "./dataset/author_ans.json"
# matchings_path = "./dataset/purchase.json"
# answers_path = "./dataset/purchase_ans.json"
dataset_name = "author"
# dataset_name = "employee"

#LLM 预测准确率
p_w = 0.917
api_use = False
# 每个token 一个
budget = 35 # 单位为 price of one token  
total_turns =20

In [129]:
from fact import FactSet
from llm_api import gpt_check
import numpy as np
import json
import time

with open(matchings_path, "r") as f:
    content = json.load(f)

### 初始化 chatgpt 的 tokenzier，用于估计correspondence 的cost func

In [130]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

### process 统计，factset 需要的数据

In [131]:
def process(content):
    import numpy as np
    c_set = content["correspondence_set"]
    # chatgpt 的tokens 的num list
    len_list = [len(encoding.encode(i[0][1]+i[1][1])) for i in c_set]
    len_list.sort()
    least_len = sum(len_list[:3])
    matchings = content["matchings"]
    Views = []
    for match in matchings:
        view = []
        for c in c_set:
            if c in match:
                view.append(1)
            else:
                view.append(0)
        Views.append(view)
    return np.array(Views), np.array(content["prob_all"]), np.array(len_list), least_len, sum(len_list), c_set

### 处理后生成 facts 

In [132]:
facts, prob, len_list, least_len, sum_c, c_set = process(content)
print("max_cost: ", least_len, f"note that budget of each round must > {least_len} and < {sum_c}:")
ex_fact = FactSet(facts=facts, prior_p=prob, ground_true=2, len_list=len_list)
random_fact = FactSet(facts=facts, prior_p=prob, ground_true=2, len_list=len_list)
brute_fact = FactSet(facts=facts, prior_p=prob, ground_true=2, len_list=len_list)
heuristic_fact = FactSet(facts=facts, prior_p=prob, ground_true=2, len_list=len_list)

max_cost:  9 note that budget of each round must > 9 and < 51:


## selector to select correspondence

### 初始化 selector

In [133]:
from query import QuerySelector, BaseQuerySelector, GreedyQuerySelector,RandomQuerySelector, HeuristicQuerySelector
 # 对应fact1, 3是0.8, 0.
query_selector = GreedyQuerySelector()
# selection_idxes, sub_facts, h = query_selector.select(ex_fact, 2, accuracy, cost_func=2)
random_selector = RandomQuerySelector()
base_selector = BaseQuerySelector()
h_selector = HeuristicQuerySelector()

## LLM答案dict
## 由于数量问题，模拟实验，从web chatgpt端，得到所有的correpspondence 验证答案

In [134]:
import json
with open(answers_path, "r") as f:
    ans_list = json.load(f)

### 参数设置

In [135]:
c_len = ex_fact.num_fact()
# 计算p(A_T) 和 P(A_T|v) 需要的LLM array:acc 
acc = np.array([[p_w for i in range(c_len)]])

In [136]:
def post_p_caculate(prior_p, p_a_v, p_a):
    return prior_p*p_a_v / p_a

####  our近似算法

In [137]:
import numpy as np
import time
approx_h_list=[ex_fact.compute_entropy()]

cost_sum = 0
turns = total_turns 
start = time.time()
while turns>0:
    selection_idxes, sub_facts, h = query_selector.select(ex_fact, budget, acc, cost_func=2)
    if api_use:
         ans = [1 if gpt_check(ix_r, c_set)=="yes" else 0 for ix_r in selection_idxes]
    else:
        ans = [1 if ans_list[ix_r]=="yes" else 0 for ix_r in selection_idxes]
    p_a,p_a_v = ex_fact.compute_ans_p(ans, selection_idxes, acc)
    p_post = ex_fact.get_prior_p()*p_a_v/p_a
    ex_fact.set_prior_p(p_post)
    approx_h_list.append(ex_fact.compute_entropy())
    turns -=1
end = time.time()

approx_timecost = end - start
print("approx:{} s".format(approx_timecost))

approx:8.27517557144165 s


## random 算法

In [138]:
all_h_list = []
cost_list = []
p_prior = random_fact.get_prior_p()
for _ in range(100):
    cost_sum_r = 0
    random_fact.set_prior_p(p_prior)
    turns_r=total_turns
    random_h_list=[random_fact.compute_entropy()]
    random_selection_idxes, _, _  =  random_selector.select(random_fact, budget, acc)
    while turns_r>0:
      
        if api_use:
            ans_r = [1 if gpt_check(ix_r, c_set)=="yes" else 0 for ix_r in random_selection_idxes]
        else:
            ans_r = [1 if ans_list[ix_r]=="yes" else 0 for ix_r in random_selection_idxes]
        p_a_r,p_a_v_r = random_fact.compute_ans_p(ans_r, random_selection_idxes, acc)
        p_post = random_fact.get_prior_p()*p_a_v / p_a
        random_fact.set_prior_p(p_post)
        random_h_list.append(random_fact.compute_entropy())
        turns_r -=1
    all_h_list.append(random_h_list)
    cost_list.append(cost_sum_r)

## Brute Algorithm

In [139]:
import numpy as np
import time

brute_h_list=[brute_fact.compute_entropy()]
cost_sum = 0
turns = total_turns 
start = time.time()
while turns>0:
    selection_idxes, sub_facts, h = base_selector.select(brute_fact, budget, acc)
    if api_use:
         ans = [1 if gpt_check(ix_r, c_set)=="yes" else 0 for ix_r in selection_idxes]
    else:
        ans = [1 if ans_list[ix_r]=="yes" else 0 for ix_r in selection_idxes]
    p_a,p_a_v = brute_fact.compute_ans_p(ans, selection_idxes, acc)
    p_post = brute_fact.get_prior_p()*p_a_v / p_a
    brute_fact.set_prior_p(p_post)
    brute_h_list.append(brute_fact.compute_entropy())
    turns -=1
end = time.time()

brute_timecost = end - start
print("brute:{} s".format(brute_timecost))

brute:1785.5654842853546 s


## Heuristic 算法

In [140]:
heuristic_h_list=[heuristic_fact.compute_entropy()]
cost_sum = 0
turns = total_turns 
start = time.time()
while turns>0:
    selection_idxes, sub_facts, h = h_selector.select(heuristic_fact, budget, acc, max_iters=10, cost_func=2)
    if api_use:
         ans = [1 if gpt_check(ix_r, c_set)=="yes" else 0 for ix_r in selection_idxes]
    else:
        ans = [1 if ans_list[ix_r]=="yes" else 0 for ix_r in selection_idxes]
    p_a,p_a_v = heuristic_fact.compute_ans_p(ans, selection_idxes, acc)
    p_post = heuristic_fact.get_prior_p()*p_a_v / p_a
    heuristic_fact.set_prior_p(p_post)
    heuristic_h_list.append(heuristic_fact.compute_entropy())
    turns -=1
end = time.time()

heuristic_timecost = end - start
print("heuristic: {}".format(heuristic_timecost))

heuristic: 219.80209803581238


approx_entropies

In [141]:
approx_h_list

[2.0179569806623436,
 0.04850071520375093,
 0.0007121559136391639,
 8.475355387646001e-06,
 9.107752459490967e-08,
 9.234666681931895e-10,
 9.018120191382035e-12,
 8.574764637194502e-14,
 7.799761819158998e-16,
 7.18871683042066e-18,
 6.543745203618148e-20,
 5.897074022670564e-22,
 5.270388787733485e-24,
 4.677591301815171e-26,
 4.12690498430247e-28,
 3.6224734142156624e-30,
 3.1655665183593934e-32,
 2.7554840150173155e-34,
 2.3902261481261242e-36,
 2.0669863201285002e-38,
 1.7825080936416507e-40]

random_entropies

In [142]:
import numpy as np
n = np.array(all_h_list)
random_h_l = n.mean(axis=0, keepdims=True)
random_h_l = random_h_l.tolist()[0]
random_h_l

[2.017956980662346,
 0.3828448384937168,
 0.36644419493071767,
 0.3662069399609961,
 0.3662041267304737,
 0.36620409653173885,
 0.36620409622571926,
 0.36620409622273153,
 0.3662040962227031,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267]

heuristic

In [143]:
heuristic_h_list

[2.0179569806623436,
 0.049749981869701455,
 0.0007123050109495532,
 8.475355408715273e-06,
 9.107752397507204e-08,
 9.234665455268117e-10,
 9.018120180378155e-12,
 8.57476463626294e-14,
 7.799761819123783e-16,
 7.188716830477893e-18,
 6.543745203618105e-20,
 5.897074022670538e-22,
 5.270388787733483e-24,
 4.6775913018151874e-26,
 4.126904984302471e-28,
 3.6224734142156736e-30,
 3.1655665183594027e-32,
 2.7554840150173146e-34,
 2.3902261481261232e-36,
 2.0669863201285e-38,
 1.7825080936416497e-40]

brute

In [144]:
brute_h_list

[2.0179569806623436,
 0.04974685119757438,
 0.0007123028671739986,
 8.475355389340578e-06,
 9.107751111090552e-08,
 9.234665444514373e-10,
 9.018120180290755e-12,
 8.574764636254668e-14,
 7.799761819081175e-16,
 7.18871683041474e-18,
 6.543745203617627e-20,
 5.897074022670519e-22,
 5.270388787733478e-24,
 4.677591301815169e-26,
 4.126904984302467e-28,
 3.62247341421566e-30,
 3.165566518359391e-32,
 2.7554840150173134e-34,
 2.3902261481261222e-36,
 2.0669863201284986e-38,
 1.7825080936416495e-40]

In [145]:
import json
dataset = "output"
def save_h_file(approx_name_f,random_f, brute_name_f, heuristic_name_f, approx_ob, random_ob, brute_ob, heuristic_ob):
    with open(f"./{dataset}/"+approx_name_f, "w") as f:
        json.dump(approx_ob, f, indent=2, ensure_ascii=False)
    
    with open(f"./{dataset}/"+random_f, "w") as w:
        json.dump(random_ob, w, indent=2, ensure_ascii=False)
        
    with open(f"./{dataset}/"+brute_name_f, "w") as f2:
        json.dump(brute_ob, f2, indent=2, ensure_ascii=False)
        
    with open(f"./{dataset}/"+ heuristic_name_f, "w") as f3:
        json.dump(heuristic_ob, f3, indent=2, ensure_ascii=False)

In [146]:
approx = {"entropy":approx_h_list, "timecost":approx_timecost, "prob":list(ex_fact.get_prior_p())}
random = {"entropy":random_h_list, "timecost":0, "prob":list(random_fact.get_prior_p())}
brute = {"entropy":brute_h_list, "timecost":brute_timecost, "prob":list(brute_fact.get_prior_p())}
heuristic = {"entropy":heuristic_h_list, "timecost":heuristic_timecost, "prob":list(heuristic_fact.get_prior_p())}

In [147]:
name = f"turns_{total_turns}" + "_budget={}_{}"
save_h_file(name.format(budget,f"{dataset_name}_approx.json"), name.format(budget,f"{dataset_name}_random.json"),
            name.format(budget, f"{dataset_name}_heuristic.json"), name.format(budget, f"{dataset_name}_brute.json"),
            approx, random, heuristic, brute)

In [148]:
11/12

0.9166666666666666